In [14]:
from sklearn import linear_model
import numpy as np
import cv2
import os


def detect_baseline(img: np.ndarray, threshold: int = 20) -> int:
    """
    detect baseline
    :param img: img to find a baseline
    :param threshold: TODO
    :returns: TODO
    """

    low = []
    for w in range(1, img.shape[1] - 1):
        if np.max(img[:, w]) > threshold:
            for h in range(img.shape[0] - 5, 0, -1):
                if img[h, w] > threshold:
                    low += [[h, w]]
                    break
    points_lower = np.array(low)

    # Robust outliers regression
    x = points_lower[:, 1].reshape(points_lower.shape[0], 1)
    y = points_lower[:, 0].reshape(points_lower.shape[0], 1)
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(x, y)
    y_mean = model_ransac.predict(np.array([img.shape[1] / 2]).reshape(1, -1))

    return int(y_mean)

In [15]:
base_dir = "symbols_dataset"
sm_dir = "sm_dig"
cap_dir = "cap"

letters = ['д', 'з', 'р', 'у', 'ф', 'ц', 'щ']

In [20]:
for i, letter in enumerate(letters):
    for j in range(160):
        letter_img = cv2.imread(os.path.join("..", base_dir, sm_dir, letter, f"{j:04d}_{letter}_00.png"), cv2.IMREAD_GRAYSCALE)
        if letter_img is None:
            continue

        try:
            if letter in ['д', 'з', 'у']:
                y = detect_baseline(255 - letter_img[:80, :])
            else:
                y = detect_baseline(255 - letter_img)
        except ValueError:
            y = 73
        letter_img = cv2.line(letter_img, (0, int(y)), (letter_img.shape[1], int(y)), (0,0,255), 2)
        cv2.imwrite(os.path.join("..", "images", f"{i}_{j}.png"), letter_img)